In [1]:
import numpy as np
from dwave.samplers import SimulatedAnnealingSampler
import clarabel

### Рюкзак
А именно *Multidimensional Multiple-Choice Quadratic Knapsack Problem* (MdMCQKP)

Пусть:
- N: количество предметов, которые можно положить в рюкзак
- M: размерность рюкзака, то есть количество различных ограничений в виде неравенств
- K: количество различных классов, на которые разбиваются предметы
- $x$: Бинарный вектор размера $N$, для индекса $i$ $x_i$ является индикатором, что был выбран предмет $i$ 

Тогда:
- profits (P): симметричная двухмерная матрица $N\times N$, где $P_{ij}$ это профит, если выбраны предметы $i$ и $j$ (для $i \not = j$: $P_{i, j}$ равен половине профита). Тогда $x^TPx$ это суммарный профит.
- groups (G): бинарная матрица размера $K \times N$, которая задает классы, для строчки $i$ будет выбран  один и только один предмет, среди которых в в соотвествующем столбце будет стоять $1$. То есть ограничение будет $\forall i \in \{1, \ldots K\}: \sum\limits_{j=1}^{N}G_{ij}x_j = 1$.
    Также можно записать как $Gx = \mathbb{1}_K$, где $\mathbb{1}_K$ - вектор из 1 размера K.
- capacity (c): вектор размера $M$, где $c_i$ равен вместимости рюкзака по измерению $i$.
- weights (W): матрица размера $M \times N$, где $W_{ij}$ равна весу(размеру) предмета $j$ для измерения рюкзака $i$.
То есть ограничение будет $\forall i \in \{1, \ldots M\}: \sum\limits_{j=1}^{N}W_{ij}x_j \le c_i$. Также можно записать это как $Wx \le c$, где неравенство имеется в виду по поокординатное.


In [4]:
# Валидирует переданные данные, возвращает (N, M, K) из описания сверху
def validatorMdMCQ(profits: np.ndarray, 
                   groups: np.ndarray, 
                   weights: np.ndarray, 
                   capacity: np.ndarray, 
                   rtol=1e-05, atol=1e-08):
    N = profits.shape[0]
    M = capacity.shape[0]
    K = groups.shape[0]
    
    if len(profits.shape) != 2:
        raise ValueError("profits is not matrix (not 2d array)")
    if len(groups.shape) != 2:
        raise ValueError("groups is not matrix (not 2d array)")
    if len(weights.shape) != 2:
        raise ValueError("weights is not matrix (not 2d array)")
    if len(capacity.shape) != 1:
        raise ValueError("capacity is not vector (not 1d array)")

    if profits.shape != (N, N):
        raise ValueError("profits is not square matrix (not (N, N) matrix)")
    if groups.shape != (K, N):
        raise ValueError("groups is not (K, N) matrix")
    if weights.shape != (M, N):
        raise ValueError("weights is not (M, N) matrix")
    
    isSymMatrix = lambda matrix: np.allclose(matrix, matrix.T, rtol=rtol, atol=atol) 
    if not isSymMatrix(profits):
        raise ValueError("profits is not symmetric matrix")
    isBinaryMatrix = lambda matrix: np.array_equal(matrix, matrix.astype(bool))
    if not isBinaryMatrix(groups):
        raise ValueError("groups is not binary matrix")
    return N, M, K

### Переводим в ADDM задачу

Сначала введем еще одно обозначение.
Пусть новая переменная $u \in \mathbb{R}_{\ge 0}^M$ и мы заменим $Wx < c$ на равенство $Wx + u = c$. Но это тоже самое, что $Wx + u - c = 0$, а это эквивалентно $\|Wx + u - c\|_2^2 \le 0$

В IV секции в статье про ADMM у нас получились обозначения:
- $q(x) = -x^TPx$ - является квадратичным
- $\mathcal{X}$ - это множество из всех бинарных векторов размерности N
- $\mathcal{U} = \mathbb{R}_{\ge 0}^M, u = u$ - множество будет выпуклым
- $\varphi(u) = 0$, оно у нас не используется
- $G = G$, $b = \mathbb{1}_K$ тут у нас обозначения совпали
- $g(x) = 0$, оно у нас не используется
- $\ell(x, u)$ заменится на $\|Wx + u - c\|_2^2$ (x и u в качестве себя) - это функция должна быть совместно выпуклой, сейчас я это не буду доказывать

Наша текущая задача 

\begin{equation}
\min_{x\in\mathcal{X}, u \in \mathbb{R}_{\ge 0}^M \subset \mathbb{R}^M} -x^TPx
\end{equation}
\begin{equation}
\text{subject to: }Gx=\mathbb{1}_K, \| Wx + u - c \|_2^2 \le 0
\end{equation}



Сделаем следующий шаг согласно статье, когда мы вводим новую переменную $z$ и ослабляем равенство $Gx=\mathbb{1}_K$. Пусть $\varrho > 0$ - какое очень большое число

\begin{equation}
\min_{x\in\mathcal{X}, z \in \mathbb{R}^N, u \in \mathbb{R}_{\ge 0}^M} -x^TPx + \dfrac{\varrho}{2}\|Gx - \mathbb{1}_K\|_2^2
\end{equation}
\begin{equation}
\text{subject to: }\| Wz + u - c \|_2^2 \le 0, x = z
\end{equation}


Теперь мы назовем вектор $\overline{x}$ размера N + M, где первые N чисел образуют вектор z, а последние M образуют вектор u, то есть мы вертикально их разместили, z над u 

Тогда:
- $f_0(x) := -x^TPx + \dfrac{\varrho}{2}\|Gx - \mathbb{1}_K\|^2_2$
- $\overline{X} := 
\left\{(z\in\mathbb{R}^N, u\in\mathbb{R}_{\ge0}^M)\Big| \|Wz + u - c\|_2^2 \le 0\right\}$
- $\iota_X(x) := 0$ если $x\in X$, иначе равна $+\infty$, это функция индикатор, которая снизу полунепрерывная (на практике это будет просто очень большое число)
- $f_1(\overline{x}) := \iota_{\overline{X}}(\overline{x})$

Тогда задача станет:

\begin{equation}
\min_{x\in\mathcal{X}, \overline{x}\in\mathbb{R}^{N+M}} f_0(x) + f_1(\overline{x})
\end{equation}
\begin{equation}
\text{subject to: }A_0x + A_1\overline{x} = 0
\end{equation}

где:
- $A_0 = E_N$ - единичная матрица размера $N\times N$
- $A_1 = [-E_N, 0_{M\times M}]$, получится матрица размера $N \times (N + M)$

Это мы свели задачу к ADMM 

In [ ]:
def knapsackSolverMdMCQ(profits: np.ndarray, groups: np.ndarray, weights: np.ndarray, capacity: np.ndarray):
    # validator raise ValueError if argument is not valid
    N, M, K = validatorMdMCQ(profits, groups, weights, capacity)
    # TODO 
    pass